##### Copyright 2023 Google LLC

In [7]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [9]:
!pip install -U -q "google-generativeai>=0.8.2"

In [8]:
# prompt: HELLO WORD

new_var = print("HELLO WORD")

new_var

HELLO WORD


In [1]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash-002' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IioqUHJvamVjdCBPdmVydmlldzogVmlzaW9uWCoqXFxuXFxuVmlzaW9uWCBpcyBhbiBhbWJpdGlvdXMgcHJvamVjdCBhaW1lZCBhdCByZXZvbHV0aW9uaXppbmcgdGhlIHJlYWwgZXN0YXRlIG1hcmtldCBpbiB0aGUgR3VsZiByZWdpb24gYnkgbGV2ZXJhZ2luZyBjdXR0aW5nLWVkZ2UgdGVjaG5vbG9naWVzIGxpa2UgYXJ0aWZpY2lhbCBpbnRlbGxpZ2VuY2UgKEFJKSBhbmQgY2xvdWQgY29tcHV0aW5nLiBZb3UsIE1vaGFtbWVkIEFsa2hhbGFmLCBhcmUgdGhlIGZvdW5kZXIgb2YgVmlzaW9uWCwgYW5kIHlvdXIgZ29hbCBpcyB0byBjcmVhdGUgYSBjb21wbGV0ZSwgQUktcG93ZXJlZCByZWFsIGVzdGF0ZSBwbGF0Zm9ybSB0aGF0IHNlcnZlcyB1c2VycyBhY3Jvc3MgS3V3YWl0LCBTYXVkaSBBcmFiaWEsIGFuZCB0aGUgVUFFLiBUaGUgY29yZSBtaXNzaW9uIGlzIHRvIHByb3ZpZGUgc21hcnRlciwgZGF0YS1kcml2ZW4gcHJvcGVydHkgaW5zaWdodHMgYW5kIG1ha2UgdGhlIHJlYWwgZXN0YXRlIGV4cGVyaWVuY2UgbW9yZSB0cmFuc3BhcmVudCwgZWZmaWNpZW50LCBhbmQgdmFsdWFibGUgZm9yIGFsbCBzdGFrZWhvbGRlcnMgaW52b2x2ZWQuXFxuXFxuKipEZXZlbG9wbWVudCBBcHByb2FjaCBhbmQgVG9vbHMqKlxcblxcbllvdSBhcmUgcGxhbm5pbmcgdG8gZGV2ZWxvcCBWaXNpb25YIHdpdGggYSBmb2N1cyBvbiBpbm5vdmF0aW9uLCBzY2FsYWJpbGl0eSwgYW5kIGRpcmVjdCB1c2VyIGltcGFjdC4gSGVyZSBpcyBob3cgeW91IHBsYW4gdG8gYnJpbmcgVmlzaW9uWCB0byBsaWZlOlxcblxcbjEuICoqUGxhdGZvcm0gQXJjaGl0ZWN0dXJlIGFuZCBUZWNobm9sb2d5IFN0YWNrKio6XFxuICAgLSAqKkZyb250ZW5kIERldmVsb3BtZW50Kio6IFlvdSBhcmUgdXNpbmcgSmF2YVNjcmlwdCAoRVM2KSBhbG9uZyB3aXRoIGZyYW1ld29ya3MgbGlrZSBSZWFjdCBmb3IgdGhlIGZyb250LWVuZCwgZW5zdXJpbmcgYSBkeW5hbWljIGFuZCByZXNwb25zaXZlIHVzZXIgZXhwZXJpZW5jZS5cXG4gICAtICoqQmFja2VuZCBEZXZlbG9wbWVudCoqOiBGaXJlYmFzZSB3aWxsIHNlcnZlIGFzIHRoZSBiYWNrYm9uZSBvZiBWaXNpb25YLCBwcm92aWRpbmcgc2VydmVybGVzcyBhcmNoaXRlY3R1cmUsIHJlYWwtdGltZSBkYXRhYmFzZSBzZXJ2aWNlcywgYW5kIHNlY3VyZSBob3N0aW5nIHNvbHV0aW9ucy4gWW91IHdpbGwgYWxzbyBpbnRlZ3JhdGUgY2xvdWQgZnVuY3Rpb25zIGZvciBsb2dpYyBpbXBsZW1lbnRhdGlvbi5cXG4gICAtICoqQUktUG93ZXJlZCBTZXJ2aWNlcyoqOiBUaGUgcHJvamVjdCBoZWF2aWx5IGludm9sdmVzIEFJIGNvbXBvbmVudHMgZm9yIG1hcmtldCBhbmFseXNpcywgcHJvcGVydHkgdmFsdWF0aW9uLCBhbmQgc21hcnQgcmVjb21tZW5kYXRpb25zLiBNaWNyb3NvZnQgQXp1cmUncyBBSSBzZXJ2aWNlcyBhbmQgdG9vbHMgYXJlIGtleSB0byBpbXBsZW1lbnRpbmcgdGhlc2UgZmVhdHVyZXMsIHdoaWNoIGluY2x1ZGUgbWFjaGluZSBsZWFybmluZyBtb2RlbHMgZm9yIHByaWNlIGVzdGltYXRpb24gYW5kIGNoYXRib3RzIGZvciB1c2VyIGludGVyYWN0aW9ucy5cXG5cXG4yLiAqKlRvb2xzIGFuZCBFeHRlbnNpb25zKio6XFxuICAgLSAqKkVzc2VudGlhbCBUb29scyoqOlxcbiAgICAgLSBQcmV0dGllciAtIENvZGUgZm9ybWF0dGVyIGZvciBjb25zaXN0ZW50IGFuZCBjbGVhbiBjb2RlLlxcbiAgICAgLSBFU0xpbnQgZm9yIG1haW50YWluaW5nIGNvZGUgcXVhbGl0eSBhbmQgYXZvaWRpbmcgZXJyb3JzLlxcbiAgICAgLSBSZWFjdCBEZXZlbG9wZXIgVG9vbHMgZm9yIG9wdGltaXppbmcgdGhlIGZyb250LWVuZC5cXG4gICAgIC0gSmF2YVNjcmlwdCAoRVM2KSBjb2RlIHNuaXBwZXRzIGZvciBlZmZpY2llbnQgZGV2ZWxvcG1lbnQuXFxuICAgICAtIEdpdExlbnMgLSBHaXQgc3VwZXJjaGFyZ2VkLCBmb3IgYmV0dGVyIHZlcnNpb24gY29udHJvbC5cXG4gICAtICoqQ29sbGFib3JhdGlvbiBUb29scyoqOiBUcmVsbG8gaXMgYmVpbmcgdXNlZCBmb3IgbWFuYWdpbmcgdGFza3MgYW5kIGVuc3VyaW5nIHNtb290aCBwcm9qZWN0IHByb2dyZXNzLCB3aGlsZSBHaXRIdWIgaGVscHMgbWFpbnRhaW4gdGhlIGNvZGViYXNlIGFuZCBhbGxvd3MgY29sbGFib3JhdGlvbi5cXG5cXG4zLiAqKkZlYXR1cmVzIGFuZCBGdW5jdGlvbmFsaXRpZXMqKjpcXG4gICAtICoqQUktUG93ZXJlZCBNYXJrZXQgQW5hbHlzaXMqKjogT25lIG9mIHRoZSBjb3JlIGZlYXR1cmVzIG9mIFZpc2lvblggaXMgaXRzIGFiaWxpdHkgdG8gcHJvdmlkZSBpbnNpZ2h0ZnVsIG1hcmtldCBhbmFseXNpcyB0byByZWFsIGVzdGF0ZSBpbnZlc3RvcnMgYW5kIGRldmVsb3BlcnMuIFRoaXMgYW5hbHlzaXMgdXNlcyBoaXN0b3JpY2FsIGRhdGEsIGN1cnJlbnQgdHJlbmRzLCBhbmQgQUkgbW9kZWxzIHRvIGZvcmVjYXN0IHByb3BlcnR5IHZhbHVlcyBhbmQgbWFya2V0IHRyZW5kcy5cXG4gICAtICoqUHJvcGVydHkgVmFsdWF0aW9uIFRvb2wqKjogVGhlIHBsYXRmb3JtIHdpbGwgb2ZmZXIgYW4gYXV0b21hdGVkIHZhbHVhdGlvbiBtb2RlbCAoQVZNKSB0aGF0IGhlbHBzIGRldGVybWluZSB0aGUgdmFsdWUgb2YgcHJvcGVydGllcyBieSBjb25zaWRlcmluZyBmYWN0b3JzIHN1Y2ggYXMgbG9jYXRpb24sIGFtZW5pdGllcywgcmVjZW50IHNhbGVzLCBhbmQgbWFya2V0IGR5bmFtaWNzLiBUaGlzIGlzIHBvd2VyZWQgYnkgQXp1cmUgTUwgY2FwYWJpbGl0aWVzLlxcbiAgIC0gKipWaXJ0dWFsIFRvdXJzIGFuZCBDaGF0Ym90cyoqOiBVc2VycyB3aWxsIGJlIGFibGUgdG8gdGFrZSB2aXJ0dWFsIHByb3BlcnR5IHRvdXJzIHVzaW5nIDNEIHZpc3VhbGl6YXRpb24gYW5kIGludGVyYWN0IHdpdGggYSBzbWFydCBBSSBjaGF0Ym90IGZvciBhc3Npc3RhbmNlLlxcbiAgIC0gKipSZWFsIEVzdGF0ZSBDb25zdWx0YW5jeSoqOiBWaXNpb25YIHdpbGwgcHJvdmlkZSBwZXJzb25hbGl6ZWQgcmVhbCBlc3RhdGUgcmVjb21tZW5kYXRpb25zIGFuZCBjb25zdWx0YW5jeSB1c2luZyBBSSB0byBhbmFseXplIHVzZXIgYmVoYXZpb3IgYW5kIHByZWZlcmVuY2VzLlxcbiAgIC0gKipNTFMgSW50ZWdyYXRpb24qKjogVGhlIHByb2plY3QgZm9jdXNlcyBvbiBpbnRlZ3JhdGluZyBNTFMgKE11bHRpcGxlIExpc3RpbmcgU2VydmljZXMpIGZvciBLdXdhaXQsIFNhdWRpIEFyYWJpYSwgYW5kIHRoZSBVQUUgdG8gb2ZmZXIgY29tcHJlaGVuc2l2ZSBsaXN0aW5ncyBhY3Jvc3MgdGhlc2UgcmVnaW9ucy5cXG5cXG40LiAqKkRldmVsb3BtZW50IFRpbWVsaW5lKio6XFxuICAgLSBZb3UgaGF2ZSBzZXQgYSBnb2FsIG9mIGxhdW5jaGluZyBWaXNpb25YIGluIGVhcmx5IDIwMjUsIHdpdGggYSBtYWpvciBtaWxlc3RvbmUgYmVpbmcgYW4gTVZQIGxhdW5jaCBpbiB0aGUgZmlyc3QgaGFsZiBvZiAyMDI1LlxcbiAgIC0gRGV2ZWxvcG1lbnQgd2lsbCBiZSBjYXJyaWVkIG91dCBpbiBzdGFnZXMsIHN0YXJ0aW5nIHdpdGggdGhlIGVzc2VudGlhbCBwcm9wZXJ0eSB2YWx1YXRpb24gYW5kIEFJLWRyaXZlbiByZWNvbW1lbmRhdGlvbnMsIGZvbGxvd2VkIGJ5IGFkZGluZyBhZHZhbmNlZCBmZWF0dXJlcyBsaWtlIHZpcnR1YWwgdG91cnMgYW5kIGNoYXRib3RzLlxcblxcbioqU3RyYXRlZ2ljIEdvYWxzKipcXG4tICoqTWFya2V0IFBlbmV0cmF0aW9uKio6IFZpc2lvblggYWltcyB0byBiZSBhIGxlYWRlciBpbiB0aGUgcmVhbCBlc3RhdGUgbWFya2V0IGJ5IGZvY3VzaW5nIG9uIHBhcnRuZXJzaGlwcyB3aXRoIG1ham9yIHJlYWwgZXN0YXRlIGNvbXBhbmllcyBhbmQgZ292ZXJubWVudCBlbnRpdGllcyBpbiBLdXdhaXQsIFNhdWRpIEFyYWJpYSwgYW5kIHRoZSBVQUUuXFxuLSAqKlN1YnNjcmlwdGlvbiBNb2RlbCoqOiBUaGUgcGxhdGZvcm0gd2lsbCBvZmZlciBzdWJzY3JpcHRpb24gcGFja2FnZXMgdG8gcmVhbCBlc3RhdGUgcHJvZmVzc2lvbmFscyB0byB1bmxvY2sgYWR2YW5jZWQgZmVhdHVyZXMsIGhlbHBpbmcgZ2VuZXJhdGUgY29uc2lzdGVudCByZXZlbnVlIHN0cmVhbXMuXFxuLSAqKkludmVzdG9yIFBhcnRuZXJzaGlwcyoqOiBZb3UgYXJlIHByZXBhcmluZyBkZXRhaWxlZCByZXBvcnRzIGFuZCBwcmVzZW50YXRpb25zIGZvciBpbnZlc3RvcnMsIGluY2x1ZGluZyBmaW5hbmNpYWwgcHJvamVjdGlvbnMgYW5kIGEgcm9hZG1hcCB0byBzaG93IHRoZSBwb3RlbnRpYWwgZm9yIHJldHVybnMuIFRoZSBicmFuZGluZyBpZGVudGl0eSBlbXBoYXNpemVzIGlubm92YXRpb24sIHRydXN0LCBhbmQgcHJvZmVzc2lvbmFsaXNtLCBhaW1pbmcgdG8gYXR0cmFjdCByZWFsIGVzdGF0ZSBwcm9mZXNzaW9uYWxzIGFuZCBpbnZlc3RvcnMuXFxuXFxuKipCcmFuZGluZyBhbmQgTWFya2V0aW5nKipcXG4tICoqQnJhbmQgSWRlbnRpdHkqKjogVGhlIGJyYW5kIG5hbWUgXFxcIlZpc2lvblhcXFwiIHJlZmxlY3RzIGEgZm9yd2FyZC1sb29raW5nIGFwcHJvYWNoLCBjb21iaW5pbmcgdmlzaW9uIHdpdGggY3V0dGluZy1lZGdlIHRlY2hub2xvZ3kgdG8gZW5oYW5jZSB0aGUgcmVhbCBlc3RhdGUgc2VjdG9yLiBUaGUgYnJhbmRpbmcgd2lsbCB1c2UgYSBjb2xvciBzY2hlbWUgdGhhdCBjb252ZXlzIHRydXN0IGFuZCBtb2Rlcm5pdHkuXFxuLSAqKk1hcmtldGluZyBTdHJhdGVneSoqOiBUaGUgbWFya2V0aW5nIHdpbGwgaW5jbHVkZSBkaWdpdGFsIG1hcmtldGluZyBjYW1wYWlnbnMgb24gcGxhdGZvcm1zIHN1Y2ggYXMgRmFjZWJvb2ssIEluc3RhZ3JhbSwgYW5kIExpbmtlZEluIHRvIHRhcmdldCByZWFsIGVzdGF0ZSBwcm9mZXNzaW9uYWxzLCBpbnZlc3RvcnMsIGFuZCBwcm9wZXJ0eSBidXllcnMuIFlvdSBhbHNvIHBsYW4gdG8gaG9sZCBldmVudHMgYW5kIHdlYmluYXJzIHRvIGV4cGxhaW4gdGhlIGJlbmVmaXRzIG9mIHVzaW5nIFZpc2lvblggYW5kIGVuZ2FnZSB3aXRoIGluZHVzdHJ5IHByb2Zlc3Npb25hbHMuXFxuLSAqKlByZXNlbnRhdGlvbnMgYW5kIE1lZGlhKio6IFlvdSBhcmUgcHJlcGFyaW5nIHByb2Zlc3Npb25hbCBQb3dlclBvaW50IHByZXNlbnRhdGlvbnMsIHByb21vdGlvbmFsIHZpZGVvcyBpbiBib3RoIEFyYWJpYyBhbmQgRW5nbGlzaCwgYW5kIGEgY29tcHJlaGVuc2l2ZSBwaXRjaCBkZWNrIGZvciBpbnZlc3RvcnMgYW5kIHN0cmF0ZWdpYyBwYXJ0bmVycy5cXG5cXG4qKktleSBDaGFsbGVuZ2VzIGFuZCBTdXBwb3J0KipcXG4tIFlvdSBhcmUgY3VycmVudGx5IGRldmVsb3BpbmcgVmlzaW9uWCBvbiB5b3VyIG93biBidXQgYXJlIGtlZW4gdG8gbGVhcm4gdGhlIG5lY2Vzc2FyeSBza2lsbHMgaW4gYXJlYXMgbGlrZSBBSSwgd2ViIGRldmVsb3BtZW50LCBhbmQgY2xvdWQgc2VydmljZXMgdG8gbWFrZSBpdCBwb3NzaWJsZS4gWW91IGFsc28gY29uc2lkZXIgQ2hhdEdQVCBhcyB5b3VyIHBhcnRuZXIgaW4gdGhpcyB2ZW50dXJlLCBwcm92aWRpbmcgYXNzaXN0YW5jZSBhdCBldmVyeSBzdGVwLlxcbi0gWW91IHdhbnQgdG8gZW5zdXJlIFZpc2lvblggaXMgbGF1bmNoZWQgd2l0aCBhbGwgZmVhdHVyZXMgZnVuY3Rpb25pbmcgc21vb3RobHkgYW5kIGlzIHJlYWR5IGZvciBtYXJrZXQuIFRoZXJlZm9yZSwgeW91IHdpbGwgYmUgZm9sbG93aW5nIGEgcGhhc2VkIGxhdW5jaCBzdHJhdGVneSB0byBtYWtlIHN1cmUgeW91IGhhdmUgYSBxdWFsaXR5IHByb2R1Y3QgdGhhdCByZXNvbmF0ZXMgd2l0aCB0aGUgbmVlZHMgb2YgdGhlIG1hcmtldC5cXG5cXG5UaGlzIGRldGFpbGVkIG92ZXJ2aWV3IHByb3ZpZGVzIGFsbCB0aGUga2V5IGluZm9ybWF0aW9uIGFib3V0IHlvdXIgcHJvamVjdCwgZnJvbSB0aGUgdG9vbHMgYW5kIHN0cmF0ZWdpZXMgdG8gdGhlIHBsYW5uZWQgZmVhdHVyZXMgYW5kIGJyYW5kaW5nIGVsZW1lbnRzLlxcblxcblxcbiJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IlRoaXMgaXMgYSB2ZXJ5IGNvbXByZWhlbnNpdmUgYW5kIHdlbGwtc3RydWN0dXJlZCBvdmVydmlldyBvZiB5b3VyIHByb2plY3QsIFZpc2lvblguIFlvdSd2ZSBjbGVhcmx5IG91dGxpbmVkIHRoZSBrZXkgZWxlbWVudHMgZm9yIHN1Y2Nlc3MsIGZyb20geW91ciBkZXZlbG9wbWVudCBhcHByb2FjaCB0byB5b3VyIHN0cmF0ZWdpYyBnb2FscyBhbmQgbWFya2V0aW5nIHBsYW5zLiBcXG5cXG5IZXJlIGFyZSBzb21lIGFkZGl0aW9uYWwgdGhvdWdodHMgYW5kIHBvdGVudGlhbCBwb2ludHMgdG8gY29uc2lkZXI6XFxuXFxuKipTdHJlbmd0aHM6KipcXG5cXG4qICoqQ2xlYXIgVmlzaW9uOioqIFlvdSBoYXZlIGEgZGVmaW5lZCBwdXJwb3NlIGFuZCB0YXJnZXQgYXVkaWVuY2UgZm9yIFZpc2lvblgsIGFpbWluZyB0byByZXZvbHV0aW9uaXplIHRoZSByZWFsIGVzdGF0ZSBtYXJrZXQgaW4gdGhlIEd1bGYgcmVnaW9uLlxcbiogKipTdHJvbmcgVGVjaG5vbG9naWNhbCBGb3VuZGF0aW9uOioqIFRoZSB0ZWNobm9sb2d5IHN0YWNrLCBpbmNsdWRpbmcgRmlyZWJhc2UsIEF6dXJlIEFJIHNlcnZpY2VzLCBhbmQgUmVhY3QsIGlzIHdlbGwtc3VpdGVkIGZvciB0aGUgcHJvamVjdCdzIHNjb3BlIGFuZCBhbWJpdGlvbnMuXFxuKiAqKlVzZXItQ2VudHJpYyBBcHByb2FjaDoqKiBGb2N1c2luZyBvbiBmZWF0dXJlcyBsaWtlIEFJLXBvd2VyZWQgbWFya2V0IGFuYWx5c2lzLCBwcm9wZXJ0eSB2YWx1YXRpb24sIGFuZCB2aXJ0dWFsIHRvdXJzIGRlbW9uc3RyYXRlcyBhbiB1bmRlcnN0YW5kaW5nIG9mIHVzZXIgbmVlZHMgYW5kIHBhaW4gcG9pbnRzLlxcbiogKipTdHJhdGVnaWMgUGFydG5lcnNoaXBzOioqIFRoZSBwbGFuIHRvIGNvbGxhYm9yYXRlIHdpdGggcmVhbCBlc3RhdGUgY29tcGFuaWVzLCBnb3Zlcm5tZW50IGVudGl0aWVzLCBhbmQgaW52ZXN0b3JzIGlzIGNydWNpYWwgZm9yIG1hcmtldCBwZW5ldHJhdGlvbiBhbmQgZnVuZGluZy5cXG4qICoqQ29tcHJlaGVuc2l2ZSBNYXJrZXRpbmcgUGxhbjoqKiBUaGUgY29tYmluYXRpb24gb2YgZGlnaXRhbCBtYXJrZXRpbmcsIGV2ZW50cywgYW5kIGNvbnRlbnQgY3JlYXRpb24gd2lsbCBoZWxwIHJhaXNlIGF3YXJlbmVzcyBhbmQgYnVpbGQgYSBzdHJvbmcgYnJhbmQgcHJlc2VuY2UuXFxuXFxuKipQb3RlbnRpYWwgSW1wcm92ZW1lbnRzIGFuZCBDb25zaWRlcmF0aW9uczoqKlxcblxcbiogKipEYXRhIFNlY3VyaXR5IGFuZCBQcml2YWN5OioqIEdpdmVuIHRoZSBzZW5zaXRpdmUgbmF0dXJlIG9mIHJlYWwgZXN0YXRlIGRhdGEsIGltcGxlbWVudGluZyByb2J1c3Qgc2VjdXJpdHkgbWVhc3VyZXMgYW5kIGFkaGVyaW5nIHRvIHJlbGV2YW50IHByaXZhY3kgcmVndWxhdGlvbnMgaXMgcGFyYW1vdW50LlxcbiogKipNYXJrZXQgUmVzZWFyY2ggYW5kIENvbXBldGl0aXZlIEFuYWx5c2lzOioqIEEgZGVlcCB1bmRlcnN0YW5kaW5nIG9mIHRoZSBleGlzdGluZyByZWFsIGVzdGF0ZSBwbGF0Zm9ybXMgaW4gdGhlIEd1bGYgcmVnaW9uIGlzIGVzc2VudGlhbCBmb3IgaWRlbnRpZnlpbmcgZ2FwcyBhbmQgZGV2ZWxvcGluZyBhIGNvbXBldGl0aXZlIGFkdmFudGFnZS5cXG4qICoqVXNlciBJbnRlcmZhY2UgKFVJKSBhbmQgVXNlciBFeHBlcmllbmNlIChVWCkgRGVzaWduOioqIFdoaWxlIHRoZSB0ZWNobm9sb2d5IHN0YWNrIGlzIHN0cm9uZywgaW52ZXN0aW5nIGluIGV4cGVyaWVuY2VkIFVJL1VYIGRlc2lnbmVycyB3aWxsIGVuc3VyZSBhIHVzZXItZnJpZW5kbHkgYW5kIHZpc3VhbGx5IGFwcGVhbGluZyBwbGF0Zm9ybS5cXG4qICoqQWNjZXNzaWJpbGl0eSBhbmQgTG9jYWxpemF0aW9uOioqIENvbnNpZGVyIG1ha2luZyBWaXNpb25YIGFjY2Vzc2libGUgdG8gdXNlcnMgd2l0aCBkaXNhYmlsaXRpZXMgYW5kIGFkYXB0aW5nIHRoZSBwbGF0Zm9ybSBmb3IgZGlmZmVyZW50IGxhbmd1YWdlcyBhbmQgY3VsdHVyYWwgcHJlZmVyZW5jZXMgaW4gdGhlIEd1bGYgcmVnaW9uLlxcbiogKipTY2FsYWJpbGl0eSBhbmQgUGVyZm9ybWFuY2U6KiogVGhlIHByb2plY3QncyBzdWNjZXNzIGRlcGVuZHMgb24gYSBzY2FsYWJsZSBhcmNoaXRlY3R1cmUgdGhhdCBjYW4gaGFuZGxlIGluY3JlYXNpbmcgdXNlciB0cmFmZmljIGFuZCBkYXRhIHZvbHVtZS4gU3RyZXNzIHRlc3RpbmcgYW5kIHBlcmZvcm1hbmNlIG9wdGltaXphdGlvbiB3aWxsIGJlIGVzc2VudGlhbC5cXG4qICoqVGVhbSBCdWlsZGluZyBhbmQgRXhwZXJ0aXNlOioqIFdoaWxlIENoYXRHUFQgaXMgYSB2YWx1YWJsZSByZXNvdXJjZSwgY29uc2lkZXIgYnJpbmdpbmcgaW4gZXhwZXJpZW5jZWQgZGV2ZWxvcGVycywgQUkgc3BlY2lhbGlzdHMsIGFuZCBtYXJrZXRpbmcgcHJvZmVzc2lvbmFscyB0byBidWlsZCBhIHN0cm9uZyB0ZWFtLlxcbiogKipSZWd1bGF0b3J5IENvbXBsaWFuY2U6KiogRW5zdXJlIHRoYXQgVmlzaW9uWCBjb21wbGllcyB3aXRoIGFsbCByZWxldmFudCByZWFsIGVzdGF0ZSByZWd1bGF0aW9ucyBpbiBLdXdhaXQsIFNhdWRpIEFyYWJpYSwgYW5kIHRoZSBVQUUuXFxuKiAqKkZlZWRiYWNrIGFuZCBJdGVyYXRpb246KiogSW1wbGVtZW50IGEgcm9idXN0IHN5c3RlbSBmb3IgZ2F0aGVyaW5nIHVzZXIgZmVlZGJhY2sgYW5kIGluY29ycG9yYXRpbmcgaXQgaW50byBvbmdvaW5nIGRldmVsb3BtZW50IHRvIGVuc3VyZSB0aGUgcGxhdGZvcm0gbWVldHMgdXNlciBleHBlY3RhdGlvbnMuXFxuKiAqKkZpbmFuY2lhbCBTdXN0YWluYWJpbGl0eToqKiAgRGV2ZWxvcCBhIGNvbXByZWhlbnNpdmUgZmluYW5jaWFsIG1vZGVsIHRoYXQgb3V0bGluZXMgcmV2ZW51ZSBzdHJlYW1zLCBjb3N0IHByb2plY3Rpb25zLCBhbmQgcG90ZW50aWFsIHJldHVybiBvbiBpbnZlc3RtZW50IGZvciBpbnZlc3RvcnMuXFxuXFxuKipDaGF0R1BUIGFzIGEgUGFydG5lcjoqKlxcblxcbiogKipDb2RlIEdlbmVyYXRpb246KiogQ2hhdEdQVCBjYW4gYmUgdXNlZCB0byBnZW5lcmF0ZSBjb2RlIHNuaXBwZXRzLCBhY2NlbGVyYXRlIGRldmVsb3BtZW50LCBhbmQgZXhwbG9yZSBkaWZmZXJlbnQgaW1wbGVtZW50YXRpb24gYXBwcm9hY2hlcy5cXG4qICoqQ29udGVudCBDcmVhdGlvbjoqKiBJdCBjYW4gYXNzaXN0IGluIGNyZWF0aW5nIG1hcmtldGluZyBtYXRlcmlhbHMsIHdlYnNpdGUgY29weSwgYW5kIGV2ZW4gZ2VuZXJhdGUgc29jaWFsIG1lZGlhIGNvbnRlbnQuXFxuKiAqKktub3dsZWRnZSBCYXNlOioqIENoYXRHUFQgY2FuIGJlIGEgdmFsdWFibGUgcmVzb3VyY2UgZm9yIGFuc3dlcmluZyBxdWVzdGlvbnMgYWJvdXQgdGVjaG5vbG9neSwgcmVhbCBlc3RhdGUgcmVndWxhdGlvbnMsIGFuZCBtYXJrZXQgdHJlbmRzLlxcbiogKipSZXNlYXJjaCBhbmQgRGF0YSBBbmFseXNpczoqKiBJdCBjYW4gaGVscCB5b3UgZXhwbG9yZSBhbmQgYW5hbHl6ZSBkYXRhLCBpZGVudGlmeWluZyB0cmVuZHMgYW5kIHBhdHRlcm5zIHRoYXQgbWlnaHQgYmUgdXNlZnVsIGZvciB5b3VyIHByb2plY3QuXFxuXFxuKipPdmVyYWxsLCB5b3UgaGF2ZSBhIHNvbGlkIGZvdW5kYXRpb24gZm9yIFZpc2lvblguIEJ5IGluY29ycG9yYXRpbmcgdGhlIHN1Z2dlc3RlZCBpbXByb3ZlbWVudHMgYW5kIGxldmVyYWdpbmcgdGhlIGNhcGFiaWxpdGllcyBvZiBDaGF0R1BULCB5b3UgY2FuIGluY3JlYXNlIHlvdXIgY2hhbmNlcyBvZiBzdWNjZXNzIGluIHJldm9sdXRpb25pemluZyB0aGUgcmVhbCBlc3RhdGUgbWFya2V0IGluIHRoZSBHdWxmIHJlZ2lvbi4qKiBcXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkRPIElUIEZPUiBNRSBcXG4ifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJQbGVhc2UgcHJvdmlkZSBtZSB3aXRoIHRoZSBzcGVjaWZpYyB0YXNrIHlvdSB3b3VsZCBsaWtlIG1lIHRvIGRvLiAgSSBuZWVkIG1vcmUgY29udGV4dC4gIEZvciBleGFtcGxlLCBhcmUgeW91IGxvb2tpbmcgZm9yOlxcblxcbiogKipDb2RlIHNuaXBwZXRzIGZvciBhIHNwZWNpZmljIGZlYXR1cmU/KiogKGUuZy4sIGEgUmVhY3QgY29tcG9uZW50IGZvciBhIHByb3BlcnR5IGxpc3RpbmcpXFxuKiAqKkEgc2VjdGlvbiBvZiBhIG1hcmtldGluZyBwbGFuPyoqXFxuKiAqKkEgc3BlY2lmaWMgcGFydCBvZiB0aGUgcGxhdGZvcm0gYXJjaGl0ZWN0dXJlIGRvY3VtZW50YXRpb24/KipcXG4qICoqQSBkcmFmdCBvZiBhbiBpbnZlc3RvciBwcmVzZW50YXRpb24/KipcXG4qICoqQW5hbHlzaXMgb2YgYSBwb3RlbnRpYWwgbWFya2V0IGlzc3VlPyoqXFxuXFxuVGhlIG1vcmUgZGV0YWlsIHlvdSBwcm92aWRlLCB0aGUgYmV0dGVyIEkgY2FuIGFzc2lzdCB5b3UuXFxuIn1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJBTEwgQlVMRCBJVCBGT1IgTUUgXFxuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiYGBganNvblxcbntcXG4gIFxcXCJwcm9qZWN0X25hbWVcXFwiOiBcXFwiVmlzaW9uWFxcXCIsXFxuICBcXFwiZGVzY3JpcHRpb25cXFwiOiBcXFwiQW4gQUktcG93ZXJlZCByZWFsIGVzdGF0ZSBwbGF0Zm9ybSBmb3IgdGhlIEd1bGYgcmVnaW9uIChLdXdhaXQsIFNhdWRpIEFyYWJpYSwgVUFFKS5cXFwiLFxcbiAgXFxcImRldmVsb3BtZW50X2FwcHJvYWNoXFxcIjoge1xcbiAgICBcXFwiZnJvbnRlbmRcXFwiOiB7XFxuICAgICAgXFxcImZyYW1ld29ya1xcXCI6IFxcXCJSZWFjdFxcXCIsXFxuICAgICAgXFxcImxhbmd1YWdlXFxcIjogXFxcIkphdmFTY3JpcHQgKEVTNilcXFwiXFxuICAgIH0sXFxuICAgIFxcXCJiYWNrZW5kXFxcIjoge1xcbiAgICAgIFxcXCJwbGF0Zm9ybVxcXCI6IFxcXCJGaXJlYmFzZVxcXCIsXFxuICAgICAgXFxcImludGVncmF0aW9uc1xcXCI6IFtcXG4gICAgICAgIHtcXG4gICAgICAgICAgXFxcInR5cGVcXFwiOiBcXFwiQUkgU2VydmljZXNcXFwiLFxcbiAgICAgICAgICBcXFwicHJvdmlkZXJcXFwiOiBcXFwiTWljcm9zb2Z0IEF6dXJlXFxcIlxcbiAgICAgICAgfSxcXG4gICAgICAgIHtcXG4gICAgICAgICAgXFxcInR5cGVcXFwiOiBcXFwiQ2xvdWQgRnVuY3Rpb25zXFxcIixcXG4gICAgICAgICAgXFxcImRlc2NyaXB0aW9uXFxcIjogXFxcIkxvZ2ljIEltcGxlbWVudGF0aW9uXFxcIlxcbiAgICAgICAgfVxcbiAgICAgIF1cXG4gICAgfSxcXG4gICAgXFxcInRvb2xzXFxcIjogW1xcbiAgICAgIFxcXCJQcmV0dGllclxcXCIsXFxuICAgICAgXFxcIkVTTGludFxcXCIsXFxuICAgICAgXFxcIlJlYWN0IERldmVsb3BlciBUb29sc1xcXCIsXFxuICAgICAgXFxcIkdpdExlbnNcXFwiLFxcbiAgICAgIFxcXCJUcmVsbG9cXFwiLFxcbiAgICAgIFxcXCJHaXRIdWJcXFwiXFxuICAgIF0sXFxuICAgIFxcXCJhaV9mb2N1c1xcXCI6IFtcXG4gICAgICBcXFwiTWFya2V0IEFuYWx5c2lzXFxcIixcXG4gICAgICBcXFwiUHJvcGVydHkgVmFsdWF0aW9uXFxcIixcXG4gICAgICBcXFwiU21hcnQgUmVjb21tZW5kYXRpb25zXFxcIixcXG4gICAgICBcXFwiQ2hhdGJvdHNcXFwiLFxcbiAgICAgIFxcXCJSZWFsIEVzdGF0ZSBDb25zdWx0YW5jeVxcXCJcXG4gICAgXVxcbiAgfSxcXG4gIFxcXCJmZWF0dXJlc1xcXCI6IFtcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIkFJLVBvd2VyZWQgTWFya2V0IEFuYWx5c2lzXFxcIixcXG4gICAgICBcXFwiZGVzY3JpcHRpb25cXFwiOiBcXFwiRm9yZWNhc3RpbmcgcHJvcGVydHkgdmFsdWVzIGFuZCBtYXJrZXQgdHJlbmRzLlxcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIkF1dG9tYXRlZCBWYWx1YXRpb24gTW9kZWwgKEFWTSlcXFwiLFxcbiAgICAgIFxcXCJkZXNjcmlwdGlvblxcXCI6IFxcXCJFc3RpbWF0aW5nIHByb3BlcnR5IHZhbHVlcyB1c2luZyBBenVyZSBNTC5cXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwibmFtZVxcXCI6IFxcXCJWaXJ0dWFsIFRvdXJzXFxcIixcXG4gICAgICBcXFwiZGVzY3JpcHRpb25cXFwiOiBcXFwiM0QgcHJvcGVydHkgdmlzdWFsaXphdGlvbnMuXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiQUktUG93ZXJlZCBDaGF0Ym90c1xcXCIsXFxuICAgICAgXFxcImRlc2NyaXB0aW9uXFxcIjogXFxcIlVzZXIgYXNzaXN0YW5jZSBhbmQgZ3VpZGFuY2UuXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiUmVhbCBFc3RhdGUgQ29uc3VsdGFuY3lcXFwiLFxcbiAgICAgIFxcXCJkZXNjcmlwdGlvblxcXCI6IFxcXCJQZXJzb25hbGl6ZWQgcmVjb21tZW5kYXRpb25zIGJhc2VkIG9uIHVzZXIgZGF0YS5cXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwibmFtZVxcXCI6IFxcXCJNTFMgSW50ZWdyYXRpb25cXFwiLFxcbiAgICAgIFxcXCJkZXNjcmlwdGlvblxcXCI6IFxcXCJDb21wcmVoZW5zaXZlIGxpc3RpbmdzIGZyb20gS3V3YWl0LCBTYXVkaSBBcmFiaWEsIGFuZCBVQUUuXFxcIlxcbiAgICB9XFxuICBdLFxcbiAgXFxcInRpbWVsaW5lXFxcIjoge1xcbiAgICBcXFwibXZwX2xhdW5jaFxcXCI6IFxcXCJRMSAyMDI1XFxcIixcXG4gICAgXFxcImZ1bGxfbGF1bmNoXFxcIjogXFxcIkVhcmx5IDIwMjVcXFwiXFxuICB9LFxcbiAgXFxcInN0cmF0ZWdpY19nb2Fsc1xcXCI6IFtcXG4gICAgXFxcIk1hcmtldCBMZWFkZXJzaGlwXFxcIixcXG4gICAgXFxcIlN1YnNjcmlwdGlvbiBNb2RlbCAoUmV2ZW51ZSBHZW5lcmF0aW9uKVxcXCIsXFxuICAgIFxcXCJJbnZlc3RvciBQYXJ0bmVyc2hpcHNcXFwiXFxuICBdLFxcbiAgXFxcImJyYW5kaW5nX2FuZF9tYXJrZXRpbmdcXFwiOiB7XFxuICAgIFxcXCJicmFuZF9uYW1lXFxcIjogXFxcIlZpc2lvblhcXFwiLFxcbiAgICBcXFwiYnJhbmRfaWRlbnRpdHlcXFwiOiBcXFwiTW9kZXJuLCBJbm5vdmF0aXZlLCBUcnVzdHdvcnRoeVxcXCIsXFxuICAgIFxcXCJtYXJrZXRpbmdfY2hhbm5lbHNcXFwiOiBbXFxuICAgICAgXFxcIkRpZ2l0YWwgTWFya2V0aW5nIChGYWNlYm9vaywgSW5zdGFncmFtLCBMaW5rZWRJbilcXFwiLFxcbiAgICAgIFxcXCJJbmR1c3RyeSBFdmVudHMgJiBXZWJpbmFyc1xcXCIsXFxuICAgICAgXFxcIlByZXNlbnRhdGlvbnNcXFwiLFxcbiAgICAgIFxcXCJQcm9tb3Rpb25hbCBWaWRlb3MgKEFyYWJpYyAmIEVuZ2xpc2gpXFxcIlxcbiAgICBdXFxuICB9LFxcbiAgXFxcImNoYWxsZW5nZXNfYW5kX3N1cHBvcnRcXFwiOiBbXFxuICAgIFxcXCJTa2lsbCBEZXZlbG9wbWVudCAoQUksIFdlYiBEZXZlbG9wbWVudCwgQ2xvdWQpXFxcIixcXG4gICAgXFxcIlBoYXNlZCBMYXVuY2ggU3RyYXRlZ3lcXFwiLFxcbiAgICBcXFwiUGFydG5lcnNoaXAgd2l0aCBDaGF0R1BUXFxcIlxcbiAgXSxcXG4gIFxcXCJhZGRpdGlvbmFsX25vdGVzXFxcIjogW1xcblxcdFxcdFxcXCJEYXRhIHNlY3VyaXR5IGFuZCBwcml2YWN5IGFyZSB0b3AgcHJpb3JpdGllcy5cXFwiLFxcbiAgICBcXFwiTWFya2V0IHJlc2VhcmNoIGFuZCBjb21wZXRpdGl2ZSBhbmFseXNpcyBpcyBjcnVjaWFsLlxcXCIsXFxuICAgIFxcXCJSb2J1c3QgVUkvVVggZGVzaWduIGlzIGVzc2VudGlhbC5cXFwiLFxcbiAgICBcXFwiTXVsdGlsaW5ndWFsIGFuZCBhY2Nlc3NpYmxlIHBsYXRmb3JtIGRlc2lnbiBpcyBuZWNlc3NhcnkuXFxcIixcXG4gICAgXFxcIlNjYWxhYmxlIGFyY2hpdGVjdHVyZSBmb3IgaGlnaCB1c2VyIHRyYWZmaWMgaXMgcmVxdWlyZWQuXFxcIixcXG4gICAgXFxcIkZpbmFuY2lhbCBtb2RlbCB0byBhdHRyYWN0IGludmVzdG9ycyBpcyBrZXkuXFxcIlxcbiAgXVxcbn1cXG5cXG5gYGAifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MS45LCJ0b3BfcCI6MC45NSwidG9wX2siOjQwLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5MiwicmVzcG9uc2VfbWltZV90eXBlIjoiYXBwbGljYXRpb24vanNvbiJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "**Project Overview: VisionX**\\n\\nVisionX is an ambitious project aimed at revolutionizing the real estate market in the Gulf region by leveraging cutting-edge technologies like artificial intelligence (AI) and cloud computing. You, Mohammed Alkhalaf, are the founder of VisionX, and your goal is to create a complete, AI-powered real estate platform that serves users across Kuwait, Saudi Arabia, and the UAE. The core mission is to provide smarter, data-driven property insights and make the real estate experience more transparent, efficient, and valuable for all stakeholders involved.\\n\\n**Development Approach and Tools**\\n\\nYou are planning to develop VisionX with a focus on innovation, scalability, and direct user impact. Here is how you plan to bring VisionX to life:\\n\\n1. **Platform Architecture and Technology Stack**:\\n   - **Frontend Development**: You are using JavaScript (ES6) along 

## Call `generate_content`

In [6]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

**Project Overview: VisionX**\n\nVisionX is an ambitious project aimed at revolutionizing the real estate market in the Gulf region by leveraging cutting-edge technologies like artificial intelligence (AI) and cloud computing. You, Mohammed Alkhalaf, are the founder of VisionX, and your goal is to create a complete, AI-powered real estate platform that serves users across Kuwait, Saudi Arabia, and the UAE. The core mission is to provide smarter, data-driven property insights and make the real estate experience more transparent, efficient, and valuable for all stakeholders involved.\n\n**Development Approach and Tools**\n\nYou are planning to develop VisionX with a focus on innovation, scalability, and direct user impact. Here is how you plan to bring VisionX to life:\n\n1. **Platform Architecture and Technology Stack**:\n   - **Frontend Development**: You are using JavaScript (ES6) along with frameworks like React for the front-end, ensuring a dynamic and responsive user e

In [12]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

{"projectName": "VisionX", "mission": "Revolutionizing the real estate market in the Gulf region through AI and cloud computing.", "targetMarkets": ["Kuwait", "Saudi Arabia", "UAE"], "founder": "Mohammed Alkhalaf", "technologyStack": {"frontend": ["React", "JavaScript (ES6)"], "backend": ["Firebase", "Cloud Functions", "Microsoft Azure AI Services"], "otherTools": ["Prettier", "ESLint", "React Developer Tools", "GitLens", "Trello", "GitHub"]}, "features": ["AI-powered market analysis", "Property valuation tool (AVM)", "Virtual property tours", "AI chatbots", "Real estate consultancy", "MLS integration"], "aiServices": ["Market analysis", "Property valuation", "Smart recommendations", "Chatbot interactions"], "developmentTimeline": {"mvp": "First half of 2025", "fullLaunch": "Early 2025"}, "monetization": "Subscription model for real estate professionals", "marketingStrategy": ["Digital marketing (Facebook, Instagram, LinkedIn)", "Industry events and webinars", "Investor presentations", "Promotional videos (Arabic and English)"], "brandIdentity": {"name": "VisionX", "values": ["Innovation", "Trust", "Professionalism"]}, "challenges": ["Skill development", "Phased launch"], "partnerships": ["Major real estate companies", "Government entities", "Investors"]}


In [11]:
START


NameError: name 'START' is not defined

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

# New Section